In [2]:
%reload_ext autoreload
from igraph import Graph
import numpy as np
#import time
#from numba import njit
from scipy.sparse import coo_matrix, dok_matrix, csr_matrix
#from collections import defaultdict
#import json
import importlib
#from __future__ import absolute_import 
import os
os.environ['PYTHONUNBUFFERED'] = '1'

In [4]:
TriangleData=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
InstrumentalData=np.zeros(8);
InstrumentalData[0]=0.5;
InstrumentalData[5]=0.5;

inflation_order=2
TriangleGraph=Graph.Formula("X->A,Y->A:B,Z->B:C,X->C")
EvansGraph=Graph.Formula("U3->A:C:D,U2->B:C:D,U1->A:B,A->C,B->D")
InstrumentalGraph=Graph.Formula("U1->X->A->B,U2->A:B")
#We are going to need to get some code to work with variable ordering.

Graph=InstrumentalGraph
Data=InstrumentalData
card=2

Graph=TriangleGraph
Data=TriangleData
card=4



In [5]:
from lib.quickgraph import LearnSomeInflationGraphParameters
#importlib.reload(sys.modules['lib.quickgraph'])
obs_count,num_vars,names=LearnSomeInflationGraphParameters(Graph,inflation_order)
[obs_count,num_vars,names]

[3, 12, ['A', 'B', 'C']]

In [6]:
from lib.inflationmatrix import InflationMatrixFromGraph,FindB
#importlib.reload(sys.modules['lib.inflationmatrix'])
SparseInflationMatrix = InflationMatrixFromGraph(Graph,inflation_order,card)

['A', 'B', 'C']


In [7]:
SparseInflationMatrix.shape

(2080, 2123776)

In [8]:
MarginalVector=FindB(Data,inflation_order)
[MarginalVector.shape,SparseInflationMatrix.shape]

[(2080,), (2080, 2123776)]

In [9]:
"""from lib.inflationlp import InflationLP
#importlib.reload(sys.modules['lib.inflationlp'])
Sol=InflationLP(TriangleSparseInflationMatrix,MarginalVector)""";

In [12]:
import compileall
import sys
import importlib
compileall.compile_file("lib\\moseklp.py",force=True)
from lib.moseklp import InfeasibilityCertificate, InfeasibilityCertificateAUTO
importlib.reload(sys.modules['lib.moseklp'])
Sol=InfeasibilityCertificateAUTO(
    SparseInflationMatrix,
    MarginalVector)

Compiling 'lib\\moseklp.py'...
Inferred primal variable count: 2123776
Inferred dual variable count: 2080
LP constructed, initiated optimizer.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2080            
  Cones                  : 0               
  Scalar variables       : 2123776         
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Simplex optimizer started.
Primal simplex optimizer started.
Primal simplex optimizer setup started.
Primal simplex optimizer setup terminated.
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 2080            
Optimizer  - Scalar variables       : 2123776           conic                  : 0               
Optimizer  - hotstart               : no              
ITER      DEGITER(%)  PFEAS       DFEAS       POBJ    

In [14]:
np.dot(Sol['x'],MarginalVector)

0.0

In [13]:
np.array(Sol['x'])

array([-0., -0., -0., ..., -0., -0., -0.])

In [ ]:
import compileall
import sys
from lib.certificate import Inequality
compileall.compile_file("lib\\certificate.py",force=True)
importlib.reload(sys.modules['lib.certificate'])
solveroutputcleanedup=Inequality(Graph,inflation_order,card,
           SparseInflationMatrix,MarginalVector,Sol)

In [15]:
solveroutputcleanedup.keys()

dict_keys(['Order of variables', 'Raw rolver output', 'Inequality as string', 'Coefficients grouped by index', 'Coefficients grouped by symbol', 'Clean solver output', 'Symolic association'])

In [17]:
np.dot(solveroutputcleanedup['Clean solver output'],MarginalVector)

-0.028238945225843096

In [14]:
solveroutputcleanedup['Coefficients grouped by symbol']

defaultdict(list,
            {'1': ['P(000)P(003)',
              'P(000)P(011)',
              'P(000)P(012)',
              'P(000)P(013)',
              'P(000)P(022)',
              'P(000)P(023)',
              'P(000)P(030)',
              'P(000)P(032)',
              'P(000)P(033)',
              'P(000)P(101)',
              'P(000)P(102)',
              'P(000)P(103)',
              'P(000)P(113)',
              'P(000)P(120)',
              'P(000)P(122)',
              'P(000)P(123)',
              'P(000)P(132)',
              'P(000)P(133)',
              'P(000)P(201)',
              'P(000)P(203)',
              'P(000)P(210)',
              'P(000)P(211)',
              'P(000)P(213)',
              'P(000)P(220)',
              'P(000)P(221)',
              'P(000)P(222)',
              'P(000)P(223)',
              'P(000)P(230)',
              'P(000)P(300)',
              'P(000)P(301)',
              'P(000)P(303)',
              'P(000)P(311)',
              'P(

In [31]:
solveroutputcleanedup['Raw rolver output']

[6.636913241209186e-13,
 0.22682586044096542,
 0.22682586044096542,
 0.22682586044096542,
 0.1504454788185834,
 -0.49999999999556133,
 0.1504454788185825,
 0.1504454788185825,
 0.11341293022048271,
 0.22682586044096542,
 0.22682586044096542,
 0.4999999999966551,
 0.15044547881858206,
 0.4999999999996523,
 0.4999999999996523,
 0.11341293022048271,
 0.22682586044096542,
 0.4999999999996523,
 0.15044547881858295,
 0.4999999999996523,
 0.4999999999966551,
 0.11341293022048271,
 0.4999999999996523,
 0.15044547881858206,
 0.4999999999966551,
 0.4999999999996523,
 0.11341293022048271,
 0.22682586044096542,
 0.22682586044096542,
 0.22682586044096542,
 6.636913241209186e-13,
 0.22682586044096542,
 0.22682586044096542,
 0.11341293022048271,
 0.22682586044096542,
 0.11341293022048271]

In [ ]:
%run lib\moseklp.py

['A', 'B', 'C']
Inferred primal variable count: 2123776
Inferred dual variable count: 2080
LP constructed, initiated optimizer.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2080            
  Cones                  : 0               
  Scalar variables       : 2123776         
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Simplex optimizer started.
Dual simplex optimizer started.
Dual simplex optimizer setup started.
Dual simplex optimizer setup terminated.
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 2080            
Optimizer  - Scalar variables       : 2123776           conic                  : 0               
Optimizer  - hotstart               : no              
ITER      DEGITER(%)  PFEAS       DFEAS       POBJ                  DOBJ   